# Instalando para usar no Colab

In [1]:
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

# Importando o Spark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

In [3]:
import findspark
findspark.init()

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.master('local[*]').appName("Challenge Alura Spark").config('spark.ui.port', '4050').getOrCreate()

In [6]:
spark

# Montando no Drive para caso for usar no Colab





In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Colocando a Spark Ui (Colab)

In [8]:
!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   


In [9]:
get_ipython().system_raw('./ngrok authtoken 2Hhcdy8qIixxWoVEa31tc8HJ4rE_2vHqKDdGAJGsM7xgYvoYk')
get_ipython().system_raw('./ngrok http 4050 &')

In [10]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://5a94-34-135-61-198.ngrok.io","proto":"http","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://5a94-34-135-61-198.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [11]:
spark

# Importando os Dados, colocando em um DataFrame

In [12]:
dados = spark.read.json('/content/drive/MyDrive/Challenge_Pyspark_Alura/Data/dataset_bruto.json')

In [13]:
linhas = dados.count()
colunas = len(dados.columns)

In [14]:
print(f'A quantidade de linhas é {linhas} e a quantidade de colunas é {colunas}')

A quantidade de linhas é 89083 e a quantidade de colunas é 3


In [15]:
# Estrutura da Base de Dados

dados.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

In [16]:
dados.show()

+--------------------+--------------------+--------------------+
|             anuncio|             imagens|             usuario|
+--------------------+--------------------+--------------------+
|{0, [], [16], [0]...|[{39d6282a-71f3-4...|{9d44563d-3405-4e...|
|{0, [], [14], [0]...|[{23d2b3ab-45b0-4...|{36245be7-70fe-40...|
|{0, [1026], [1026...|[{1da65baa-368b-4...|{9dc415d8-1397-4d...|
|{0, [120], [120],...|[{79b542c6-49b4-4...|{9911a2df-f299-4a...|
|{0, [3], [3], [0]...|[{e2bc497b-6510-4...|{240a7aab-12e5-40...|
|{0, [20], [15], [...|[{2de09d46-dc0d-4...|{3c7057f5-0923-42...|
|{3, [43], [43], [...|[{147a80d9-cd40-4...|{5a9736b5-aaa0-4a...|
|{2, [42], [42], [...|[{35740004-063d-4...|{ec48d96a-137c-49...|
|{0, [], [12], [0]...|[{6d3d2aec-c96f-4...|{dad7db63-e19c-44...|
|{1, [41], [41], [...|[{3d404069-418e-4...|{a845f35f-3ab3-46...|
|{3, [43], [43], [...|[{5b8a8de7-2088-4...|{8c8de313-45d5-4c...|
|{0, [10], [4], [0...|[{5385da42-8276-4...|{0b1d4ee0-2bc3-4c...|
|{0, [12], [12], [...|[{7

# Separando os Dados das Colunas e deixando apenas a coluna Anuncio (Porque as outras não são necessárias)

In [17]:
dados = dados.select('anuncio.*')

In [18]:
dados.show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|        []|     [16]|      [0]|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|    [0]|   [0]|       Usado|      Outros|  Comercial| [1]|[{260, 107, Venda...|
|    0|        []|     [14]|      [0]|                  []|{Centro, 20051040...|b6ffbae1-17f6-487...|    [0]|    []|       Usado|      Outros|  Comercial| [0]|[{260, 107, Venda...|
|    0|    [1026]|   [1026]|      [0]|                  []|{Maria da Graça, ...|1fb030a5-9e3e-4

# Filtrar a base de dados (tipo_uso, tipo_unidade e tipo_anuncio)

In [19]:
dados.groupby(['tipo_uso', 'tipo_unidade', 'tipo_anuncio']).count().orderBy('count', ascending=False).show()

+-----------+------------+------------+-----+
|   tipo_uso|tipo_unidade|tipo_anuncio|count|
+-----------+------------+------------+-----+
|Residencial| Apartamento|       Usado|66562|
|Residencial|        Casa|       Usado|10224|
|Residencial|      Outros|       Usado| 7502|
|  Comercial|      Outros|       Usado| 4443|
|Residencial| Apartamento|  Lançamento|  235|
|  Comercial|        Casa|       Usado|   92|
|Residencial|      Outros|  Lançamento|   15|
|  Comercial| Apartamento|       Usado|    4|
|  Comercial|      Outros|  Lançamento|    3|
|Residencial|        Casa|  Lançamento|    3|
+-----------+------------+------------+-----+



In [20]:
dados\
  .select('tipo_uso')\
  .groupBy('tipo_uso')\
  .count()\
  .show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|  Comercial| 4542|
|Residencial|84541|
+-----------+-----+



In [21]:
dados\
    .select('tipo_unidade')\
    .groupBy('tipo_unidade')\
    .count()\
    .show()

+------------+-----+
|tipo_unidade|count|
+------------+-----+
|      Outros|11963|
| Apartamento|66801|
|        Casa|10319|
+------------+-----+



In [22]:
dados\
    .select('tipo_anuncio')\
    .groupBy('tipo_anuncio')\
    .count()\
    .show()

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|88827|
|  Lançamento|  256|
+------------+-----+



In [23]:
freq = dados\
  .select('*')\
  .where('(tipo_uso=="Residencial")')\
  .where('tipo_unidade=="Apartamento"')\
  .where('tipo_anuncio=="Usado"')

In [24]:
freq.count()

66562

# Transformando as colunas dos cômodos dos imóveis de listas para inteiros (quartos, suites, banheiros, vaga, area_total e area_util)

In [25]:
from pyspark.sql import functions as f
from pyspark.sql.types import ArrayType,IntegerType

In [26]:
dados.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area_util: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- banheiros: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- endereco: struct (nullable = true)
 |    |-- bairro: string (nullable = true)
 |    |-- cep: string (nullable = true)
 |    |-- cidade: string (nullable = true)
 |    |-- estado: string (nullable = true)
 |    |-- latitude: double (nullable = true)
 |    |-- longitude: double (nullable = true)
 |    |-- pais: string (nullable = true)
 |    |-- rua: string (nullable = true)
 |    |-- zona: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- suites: array (nullable = true)
 |    |-- element: long (c

In [27]:
listas_int = ['quartos', 'suites', 'banheiros', 'vaga', 'area_total', 'area_util']

In [28]:
for col in listas_int:
    dados\
    .select(f.size(f.col(col)).alias((col)))\
    .groupby(col)\
    .count()\
    .show()

+-------+-----+
|quartos|count|
+-------+-----+
|      1|88120|
|      2|  160|
|      0|  803|
+-------+-----+

+------+-----+
|suites|count|
+------+-----+
|     1|79431|
|     2|  103|
|     0| 9549|
+------+-----+

+---------+-----+
|banheiros|count|
+---------+-----+
|        1|88798|
|        2|  112|
|        0|  173|
+---------+-----+

+----+-----+
|vaga|count|
+----+-----+
|   1|84037|
|   2|   94|
|   0| 4952|
+----+-----+

+----------+-----+
|area_total|count|
+----------+-----+
|         1|76488|
|         0|12595|
+----------+-----+

+---------+-----+
|area_util|count|
+---------+-----+
|        1|88846|
|        2|  224|
|        0|   13|
+---------+-----+



In [42]:
dados.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area_util: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- banheiros: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- quartos: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- suites: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- 

In [51]:
dados = dados.select([f.col(c)[0].alias(c) if c in listas_int else c for c in dados.columns])

# Deixando apenas as colunas Bairro e Zona no DataFrame 

In [29]:
dados = dados\
        .withColumn('bairro', dados['endereco']['bairro'])\
        .withColumn('zona', dados['endereco']['zona'])\
        .drop('endereco')
dados.show()

+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------+------------+
|andar|area_total|area_util|banheiros|     caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|        bairro|        zona|
+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------+------------+
|    0|        []|     [16]|      [0]|                  []|47d553e0-79f2-4a4...|    [0]|   [0]|       Usado|      Outros|  Comercial| [1]|[{260, 107, Venda...|        Centro|Zona Central|
|    0|        []|     [14]|      [0]|                  []|b6ffbae1-17f6-487...|    [0]|    []|       Usado|      Outros|  Comercial| [0]|[{260, 107, Venda...|        Centro|Zona Central|
|    0|    [1026]|   [1026]|      [0]|                  []|1

# Transformando cada campo da coluna "valores" em uma coluna separada

In [30]:
dados = dados\
        .withColumn('condominio', dados.valores.condominio[0])\
        .withColumn('iptu', dados.valores.iptu[0])\
        .withColumn('tipo', dados.valores.tipo[0])\
        .withColumn('valor', dados.valores.valor[0])\
        .drop(dados.valores)

In [31]:
dados.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area_util: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- banheiros: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- quartos: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- suites: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- 

In [32]:
dados.show()

+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------+------------+----------+----+-----+-----+
|andar|area_total|area_util|banheiros|     caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|        bairro|        zona|condominio|iptu| tipo|valor|
+-----+----------+---------+---------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------+------------+----------+----+-----+-----+
|    0|        []|     [16]|      [0]|                  []|47d553e0-79f2-4a4...|    [0]|   [0]|       Usado|      Outros|  Comercial| [1]|        Centro|Zona Central|       260| 107|Venda|10000|
|    0|        []|     [14]|      [0]|                  []|b6ffbae1-17f6-487...|    [0]|    []|       Usado|      Outros|  Comercial| [0]|        Centro|Zona Central|       260| 107|Venda|10000|
|    0|    [1026]|   [102

# Selecionando apenas os valores de Venda.

In [33]:
dados = dados\
            .select('*')\
            .where('tipo=="venda"')

# Salvando os dados no formato parquet 

In [34]:
dados.write.parquet(
    path='/content/drive/MyDrive/Challenge_Pyspark_Alura/parquet',
    mode='overwrite'
)

In [35]:
teste_parquet = spark.read.parquet(
    '/content/drive/MyDrive/Challenge_Pyspark_Alura/parquet'
  )

In [36]:
teste_parquet.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- area_util: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- banheiros: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- caracteristicas: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- quartos: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- suites: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- 

# Salvando os dados em CSV

In [37]:
from pyspark.sql.types import StringType

In [46]:
dados = dados.withColumn('caracteristicas', dados['caracteristicas'].cast(StringType()))
dados.select('caracteristicas').printSchema()

root
 |-- caracteristicas: string (nullable = true)



In [49]:
dados = dados.drop('endereco')
dados = dados.drop('valores')

In [52]:
dados.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: string (nullable = true)
 |-- area_util: string (nullable = true)
 |-- banheiros: long (nullable = true)
 |-- caracteristicas: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: long (nullable = true)
 |-- suites: long (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: long (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)
 |-- condominio: string (nullable = true)
 |-- iptu: string (nullable = true)
 |-- tipo: string (nullable = true)
 |-- valor: string (nullable = true)



In [53]:
dados.write.csv(
    path='/content/drive/MyDrive/Challenge_Pyspark_Alura/csv',
    mode='overwrite',
    sep=';',
    header=True,
)

# Comparando o Desempenho de Leitura

In [54]:
%time

parquet = spark.read.parquet('/content/drive/MyDrive/Challenge_Pyspark_Alura/parquet')
parquet.show()

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.11 µs
+-----+----------+---------+---------+---------------+---+-------+------+------------+------------+--------+----+------+----+----------+----+----+-----+
|andar|area_total|area_util|banheiros|caracteristicas| id|quartos|suites|tipo_anuncio|tipo_unidade|tipo_uso|vaga|bairro|zona|condominio|iptu|tipo|valor|
+-----+----------+---------+---------+---------------+---+-------+------+------------+------------+--------+----+------+----+----------+----+----+-----+
+-----+----------+---------+---------+---------------+---+-------+------+------------+------------+--------+----+------+----+----------+----+----+-----+



In [55]:
%time

csv = spark.read.csv('/content/drive/MyDrive/Challenge_Pyspark_Alura/csv')
csv

CPU times: user 5 µs, sys: 0 ns, total: 5 µs
Wall time: 9.78 µs


DataFrame[_c0: string]

In [56]:
spark.stop()